Finalmente, hagan otro archivo de código para el cómputo del modelo global. Primero usen FedAvg, pero también incluyan otras dos formas de computarlo (investiga, explica brevemente y presenta una implementación).

In [1]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, classification_report
from TheModel import build 

# cargar los modelos locales entrenados en local_training.ipynb
loaded_local_models = [
    tf.keras.models.load_model(os.path.join(".", file))
    for file in sorted(os.listdir(".")) if file.startswith("lmodel") and file.endswith(".keras")
]
#  conjunto de entrenamient
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = np.expand_dims(x_train / 255.0, -1)
x_test = np.expand_dims(x_test / 255.0, -1)# normalización y expansion de dimension

# obtener los pesos de cada modelo 
local_weights = [m.get_weights() for m in loaded_local_models]
local_sizes = [12000] * len(loaded_local_models)  

# conjunto de prueba de MNIST
(_, _), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_test = np.expand_dims(x_test / 255.0, -1)  # normalización y expansion de dimension

# seleccionar un modelo base para la arquitectura
base_model = loaded_local_models[0]

# función para evaluar un modelo con pesos 
def evaluate_model(weights, base_model, x_test, y_test):
    model = tf.keras.models.clone_model(base_model)  # clona la arquitectura 
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.set_weights(weights)  # asigna los pesos agregados

    #  predicciones y métricas
    y_pred = model.predict(x_test, verbose=0)
    y_pred_classes = np.argmax(y_pred, axis=1)
    acc = accuracy_score(y_test, y_pred_classes)
    f1 = f1_score(y_test, y_pred_classes, average='weighted')

    print(classification_report(y_test, y_pred_classes))


# FEDAVG

El servidor central agrega los modelos recibidos de los clientes promediando sus parámetros. Este proceso de promediación garantiza que el modelo global aproveche la información obtenida de los diferentes clientes, preservando al mismo tiempo la privacidad. Es simple, eficiente y funciona bien cuando los datos están balanceados

https://how.dev/answers/what-is-federated-averaging-fedavg


In [2]:
def fed_avg(weights):
    return [np.mean(np.array(w), axis=0) for w in zip(*weights)]

fedavg_weights = fed_avg(local_weights)
evaluate_model(fedavg_weights, base_model, x_test, y_test)

              precision    recall  f1-score   support

           0       1.00      0.19      0.32       980
           1       0.38      0.99      0.55      1135
           2       0.95      0.04      0.07      1032
           3       0.59      0.24      0.35      1010
           4       1.00      0.00      0.00       982
           5       0.73      0.45      0.56       892
           6       1.00      0.01      0.03       958
           7       0.50      0.58      0.53      1028
           8       0.28      0.81      0.42       974
           9       0.39      0.72      0.51      1009

    accuracy                           0.41     10000
   macro avg       0.68      0.40      0.33     10000
weighted avg       0.68      0.41      0.34     10000



# FED WEIGHTED AVG

Es una extensión de FedAvg al ponderar cada modelo local según la cantidad de datos que usó para entrenarse. Esto hace que modelos con mayor volumen de datos tengan mayor influencia en el modelo global. 

In [3]:
def fed_weighted_avg(weights, sizes):
    total = sum(sizes)
    return [
        np.sum([w * (s / total) for w, s in zip(layer, sizes)], axis=0)
        for layer in zip(*weights)
    ]
fedweighted_weights = fed_weighted_avg(local_weights, local_sizes)
evaluate_model(fedweighted_weights, base_model, x_test, y_test)

              precision    recall  f1-score   support

           0       1.00      0.19      0.32       980
           1       0.38      0.99      0.55      1135
           2       0.95      0.04      0.07      1032
           3       0.59      0.24      0.35      1010
           4       1.00      0.00      0.00       982
           5       0.73      0.45      0.56       892
           6       1.00      0.01      0.03       958
           7       0.50      0.58      0.53      1028
           8       0.28      0.81      0.42       974
           9       0.39      0.72      0.51      1009

    accuracy                           0.41     10000
   macro avg       0.68      0.40      0.33     10000
weighted avg       0.68      0.41      0.34     10000



# FED MEDIAN

Calcula la mediana de los pesos de cada capa entre todos los modelos. Su ventaja es la tolerancia a valores extremos pero puede afectar la precisión considerablemente.

In [5]:
def fed_median(weights):
    return [np.median(np.array(w), axis=0) for w in zip(*weights)]

fedmedian_weights = fed_median(local_weights)
evaluate_model( fedmedian_weights, base_model, x_test, y_test)

              precision    recall  f1-score   support

           0       0.93      0.14      0.25       980
           1       0.53      0.97      0.68      1135
           2       0.63      0.02      0.04      1032
           3       0.00      0.00      0.00      1010
           4       1.00      0.00      0.01       982
           5       0.55      0.46      0.50       892
           6       1.00      0.00      0.01       958
           7       0.60      0.11      0.18      1028
           8       0.14      0.78      0.24       974
           9       0.36      0.55      0.43      1009

    accuracy                           0.31     10000
   macro avg       0.57      0.30      0.23     10000
weighted avg       0.57      0.31      0.24     10000



# FED MAX

Toma el valor máximo por posición en cada capa de pesos entre los modelos locales. Es una estrategia experimental y se ha usado para estudiar comportamientos extremos

In [6]:
# FED MAX: Máximo valor por capa (experimental, no recomendable en la práctica)
def fed_max(weights):
    return [np.max(np.array(w), axis=0) for w in zip(*weights)]

# Aplicamos la agregación por máximo
fedmax_weights = fed_max(local_weights)

# Evaluamos el modelo resultante
evaluate_model( fedmax_weights, base_model, x_test, y_test)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       980
           1       0.00      0.00      0.00      1135
           2       0.00      0.00      0.00      1032
           3       0.00      0.00      0.00      1010
           4       0.00      0.00      0.00       982
           5       0.09      1.00      0.16       892
           6       0.00      0.00      0.00       958
           7       0.51      0.04      0.07      1028
           8       0.00      0.00      0.00       974
           9       0.00      0.00      0.00      1009

    accuracy                           0.09     10000
   macro avg       0.06      0.10      0.02     10000
weighted avg       0.06      0.09      0.02     10000



/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Métodos con múltiples rondas

En cada ronda se entrena localmente, se agregan los resultados y se actualiza el modelo global, que luego vuelve a enviarse a los clientes para comenzar una nueva ronda de entrenamiento. Cada ronda mejora el modelo global a partir de nuevos ajustes locales. Con suficientes rondas y buena agregación se puede alcanzar rendimiento cercano a entrenar un único modelo con todos los datos juntos

## TRIMMED MEAN

Trimmed Mean descarta una fracción de los valores más altos y más bajos en cada posición de los pesos antes de calcular el promedio. Así evita que valores atípicos (por ruido o errores de entrenamiento) influyan en la agregación. 
https://www.investopedia.com/terms/t/trimmed_mean.asp

In [7]:
def fed_trimmed_mean(weights, trim_ratio=0.2):
    trimmed_weights = []

    # itera capa por capa
    for layer in zip(*weights):
        stacked = np.stack(layer)  #  convierte a array 

        # calcula cuántos extremos quitar
        k = int(trim_ratio * len(stacked))

        # ordena por posición
        sorted_w = np.sort(stacked, axis=0)

        # quita los valores más bajos y más altos
        trimmed = sorted_w[k:-k] if k > 0 else stacked

        # promedia los valores que quedan
        trimmed_weights.append(np.mean(trimmed, axis=0))

    return trimmed_weights

def federated_rounds_trimmed(local_weights, base_model, rounds=3, trim_ratio=0.2):
    # inicializa modelo global con arquitectura base
    global_model = tf.keras.models.clone_model(base_model)
    global_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # pesos con agregación trimmed mean
    global_model.set_weights(fed_trimmed_mean(local_weights, trim_ratio))

    for _ in range(rounds):
        updated_weights = []

        # entrenamiento local de cada cliente
        for weights in local_weights:
            local_model = tf.keras.models.clone_model(base_model)
            local_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

            # usa los pesos globales actuales como base
            local_model.set_weights(global_model.get_weights())

            # entrena con subconjunto simulado de datos
            local_model.fit(x_train[:12000], y_train[:12000], epochs=1, batch_size=32, verbose=0)

            # guarda pesos entrenados
            updated_weights.append(local_model.get_weights())

        # agrega pesos usando otra vez con trimmed mean
        global_model.set_weights(fed_trimmed_mean(updated_weights, trim_ratio))

    return global_model.get_weights()

trimmed_weights = federated_rounds_trimmed(local_weights, base_model)
evaluate_model(trimmed_weights, base_model, x_test, y_test)

              precision    recall  f1-score   support

           0       0.98      0.99      0.99       980
           1       0.99      0.99      0.99      1135
           2       0.97      0.99      0.98      1032
           3       0.99      0.99      0.99      1010
           4       0.99      0.98      0.99       982
           5       0.99      0.98      0.98       892
           6       0.99      0.98      0.99       958
           7       0.98      0.97      0.98      1028
           8       0.97      0.98      0.97       974
           9       0.98      0.98      0.98      1009

    accuracy                           0.98     10000
   macro avg       0.98      0.98      0.98     10000
weighted avg       0.98      0.98      0.98     10000



##  FED WEIGHTED AVG

In [8]:
def federated_rounds_weighted(local_weights, base_model, local_sizes, rounds=3):
    global_model = tf.keras.models.clone_model(base_model)
    global_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    global_model.set_weights(fed_weighted_avg(local_weights, local_sizes))

    for _ in range(rounds):
        updated_weights = []
        for weights in local_weights:
            local_model = tf.keras.models.clone_model(base_model)
            local_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
            local_model.set_weights(global_model.get_weights())
            local_model.fit(x_train[:12000], y_train[:12000], epochs=1, batch_size=32, verbose=0)
            updated_weights.append(local_model.get_weights())

        global_model.set_weights(fed_weighted_avg(updated_weights, local_sizes))

    return global_model.get_weights()

fedweighted_multi_weights = federated_rounds_weighted(local_weights, base_model, local_sizes)
evaluate_model(fedweighted_multi_weights, base_model, x_test, y_test)

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       980
           1       0.99      0.99      0.99      1135
           2       0.98      0.98      0.98      1032
           3       0.99      0.99      0.99      1010
           4       0.99      0.97      0.98       982
           5       0.99      0.98      0.98       892
           6       0.98      0.99      0.99       958
           7       0.97      0.98      0.98      1028
           8       0.98      0.97      0.98       974
           9       0.97      0.98      0.97      1009

    accuracy                           0.98     10000
   macro avg       0.98      0.98      0.98     10000
weighted avg       0.98      0.98      0.98     10000



## FEDAVG

In [ ]:
def federated_rounds_fedavg(local_weights, base_model, rounds=3):
    global_model = tf.keras.models.clone_model(base_model)
    global_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    global_model.set_weights(fed_avg(local_weights))  # inicializa con FedAvg

    for _ in range(rounds):
        updated_weights = []
        for weights in local_weights:

            local_model = tf.keras.models.clone_model(base_model)
            local_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
            local_model.set_weights(global_model.get_weights())
            local_model.fit(x_train[:12000], y_train[:12000], epochs=1, batch_size=32, verbose=0)
            updated_weights.append(local_model.get_weights())

        # nueva agregación
        global_model.set_weights(fed_avg(updated_weights))

    return global_model.get_weights()

fedavg_multi_weights = federated_rounds_fedavg(local_weights, base_model)
evaluate_model(fedavg_multi_weights, base_model, x_test, y_test)

              precision    recall  f1-score   support

           0       0.98      0.99      0.99       980
           1       0.98      1.00      0.99      1135
           2       0.98      0.98      0.98      1032
           3       0.98      0.99      0.98      1010
           4       0.99      0.99      0.99       982
           5       0.99      0.97      0.98       892
           6       0.99      0.98      0.99       958
           7       0.97      0.98      0.98      1028
           8       0.98      0.97      0.97       974
           9       0.99      0.97      0.98      1009

    accuracy                           0.98     10000
   macro avg       0.98      0.98      0.98     10000
weighted avg       0.98      0.98      0.98     10000



# Conclusión

Al evaluar los diferentes métodos nos dimos cuenta que FedMedian y FedMax fueron los peores porque son buenos cuando hay datos extrmos ruidosos o asi, pero en nuestros datos de mnist que son limpios y balanceados, creemos que descartaron información útil. FedAvg y FedWeightedAvg funcionaron mejor que los otros dos porque probablemente toman una representación más distribuida y recopilan mejor la información. Pero definitivamente los modelos mejoraron considerablemente al aplicar múltiples rondas de entrenamiento federado, ya que el modelo global se actualiza iterativamente y se mejora con la retroalimentación de los clientes.